In [1]:
# compenency query is for fasc diam, nerv diam, and lateriality

In [2]:
import pandas as pdfdsadaasdfd
import uuid

from sqlalchemy import inspect
from sqlalchemy.orm import Session
from sqlalchemy.inspection import inspect
from sqlalchemy.exc import IntegrityError

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from sqlalchemy.sql.elements import ClauseElement
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    Units,
    Addresses,
    Aspects,
    DescriptorsInst,
    DescriptorsQuant,
    ObjDescInst,
    ObjDescQuant,
    ValuesInst,
    ValuesQuant,
)

session = get_session(echo=False)

object_ = Objects(
    id="55c5b69c-test-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# ingest = Ingest(session=session)
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test3",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
)

print(list(desc_quant.__mapper__.relationships)[0].direction)
getattr(desc_quant, "aspects", [])

RelationshipDirection.MANYTOONE


/Users/tmsincomb/Dropbox (Personal)/repos/quantitative-db/quantdb/ingest_tabular/models.py:1487: SAWarning: relationship 'ObjDescCat.values_cat' will copy column obj_desc_cat.desc_cat to column values_cat.desc_cat, which conflicts with relationship(s): 'DescriptorsCat.values_cat' (copies descriptors_cat.id to values_cat.desc_cat). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="values_cat"' to the 'ObjDescCat.values_cat' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operatio

In [3]:
def object_as_dict(obj):
    return {
        c.key: getattr(obj, c.key)
        for c in inspect(obj).mapper.column_attrs
        if getattr(obj, c.key)
    }


def get_or_create(session, obj, back_populate=None):
    model = obj.__class__
    data = object_as_dict(obj)
    instance = session.query(model).filter_by(**data).one_or_none()
    if instance:
        return instance
    else:
        params = {
            k: v for k, v in data.items() if not isinstance(v, ClauseElement)
        }
        instance = model(**params)
        try:
            session.add(instance)
            session.commit()
            if back_populate:
                for attr, value in back_populate.items():
                    setattr(instance, attr, value)
                session.commit()
        except Exception:
            session.rollback()
            instance = session.query(model).filter_by(**data).one()
            return instance
        else:
            return instance


def back_populate_tables(db: Session, obj) -> object:
    """
    Recursively back-populates objects in the database,
    retrieving existing objects instead of failing on duplicates.
    Iterates through all relationships of the object for both
    initial population and recursive calls.
    Includes a complete rollback if any commit fails.
    Filters by all primary key columns of the object.
    Handles cases where the child object is not a valid ORM instance.

    Args:
        db: The database session.
        obj: The ORM model object.

    Returns:
        The ORM model object, with its related objects back-populated.
    """

    mapper = obj.__mapper__  # Get the mapper for the object
    print(obj)
    try:
        # Iterate through all relationships of the object
        for relationship_prop in mapper.relationships:
            if (
                relationship_prop.direction.name == "MANYTOONE"
                or relationship_prop.direction.name == "MANYTOMANY"
            ):
                parent = getattr(obj, relationship_prop.key)
                if parent:
                    filter_criteria = object_as_dict(parent)
                    existing_obj = (
                        db.query(type(parent))
                        .filter_by(**filter_criteria)
                        .first()
                    )
                    if existing_obj:
                        for key, value in object_as_dict(existing_obj).items():
                            if hasattr(parent, key):
                                setattr(parent, key, value)
                    print("populated", object_as_dict(parent))
                    setattr(obj, relationship_prop.key, parent)
                    # db.add(parent)
                    # db.commit()
                    back_populate_tables(db, parent)

        print("object", type(obj))
        filter_criteria = object_as_dict(obj)
        print("filter using", filter_criteria)
        existing_obj = db.query(type(obj)).filter_by(**filter_criteria).first()
        print("is existing?", existing_obj)
        if not existing_obj:
            db.merge(obj)
            db.flush()
            db.commit()
            # db.refresh(obj)
            print("after commit", object_as_dict(obj))
        else:
            for key, value in object_as_dict(existing_obj).items():
                if hasattr(obj, key):
                    setattr(obj, key, value)
            print("after update", object_as_dict(existing_obj))

    except IntegrityError as e:
        db.rollback()  # Rollback the transaction in case of an error
        print(f"Error during commit: {e}")
        raise  # Re-raise the exception to indicate failure

    return obj  # Return the object with its related objects back-populated

In [4]:
# Params
object = Objects(
    id="N:dataset:99a9a99a-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# back_populate_tables(session, object)

In [5]:
# Domain
unit = Units(
    label="test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/test",
)

aspect = Aspects(
    label="diameter-test-backpop4",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4",
)

desc_inst = DescriptorsInst(
    label="nerve-volume-test",
    iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    description=None,
)

desc_quant = DescriptorsQuant(
    shape="scalar",
    label="nerve-volume-test8",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)


# getattr(desc_quant, "aspects")
# list(desc_quant.aspects.__mapper__.relationships)
# back_populate_tables(session, desc_quant)
# print("here", aspect.id)
# print("here", desc_quant.aspects)

In [6]:
# Objects

address = Addresses(
    addr_type="constant",
    value_type="single",
)

obj_desc_inst = ObjDescInst(
    objects=object,
    descriptors_inst=desc_inst,
    addresses_field=address,
)

obj_desc_quant = ObjDescQuant(
    objects=object,
    descriptors_quant=desc_quant,
    addresses_field=address,
)

back_populate_tables(session, obj_desc_inst)
back_populate_tables(session, obj_desc_quant)

populated {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
object <class 'quantdb.ingest_tabular.models.Addresses'>
filter using {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
is existing? <quantdb.ingest_tabular.models.Addresses object at 0x156bea750>
after update {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
populated {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter using {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x156c41df0>
after update {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.i

In [7]:
# Value
values_inst = ValuesInst(
    type="sample",
    id_sub="sub-test",
    id_formal="sam-l-seg-c7-A-level-test",
    local_identifier=None,
    id_sam="sam-l-seg-c7-A-level-test",
    # descriptors_inst=desc_inst,
    # dataset=object.id,
    objects=object,
)
values_quant = ValuesQuant(
    value=0.5,
    objects=object,
    orig_value=None,
    orig_units=None,
    descriptors_inst=desc_inst,
    descriptors_quant=desc_quant,
    values_inst=values_inst,
    value_blob={"key1": "value1"},
)

with session.no_autoflush:
    back_populate_tables(session, values_quant)

populated {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter using {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
is existing? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x156c7a810>
after update {'id': 34, 'label': 'nerve-volume-test', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test'}
populated {'id': 34, 'shape': 'scalar', 'label': 'nerve-volume-test8', 'aggregation_type': 'instance', 'aspect': 29, 'domain': 34}
populated {'id': 29, 'label': 'diameter-test-backpop4', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/diameter-test-backpop4'}
object <class 'quantdb.ingest_tabular.models.Aspects'>
filter using {'id': 29, 'label': 'diameter-test-backpop4', 'iri': 'http://uri.interlex.org/tgbugs/

In [8]:
# ingest.insert_objects(
#     objects=[
#         Objects(id="55c5b69c-a5b8-4881-a105-e4048af26fa5", id_type="dataset"),
#     ]
# )
import pandas as pd

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df = df.T.reset_index(drop=True)
df["id_sub"] = "sub-" + df.sub_sam
df.index = df.index + 1
df.head()

,NFasc,dFasc_um,dNerve_um,laterality,level,sex,sub_sam,thkperi_um,id_sub
1,15,"[437.21394287682614, 258.82574913412543, 307.9...",2546.924091677267,L,C,M,C50-2,"[24.794722747116708, 27.92527295706188, 17.016...",sub-C50-2
2,5,"[120.12238286446609, 243.86502700380925, 322.2...",2019.0831006899302,L,C,M,C53-2,"[23.91547828345668, 15.931285254737446, 29.499...",sub-C53-2
3,8,"[365.4037755654016, 284.53752037921583, 53.536...",1615.1166896226846,L,C,M,C54-3,"[19.700748058232335, 28.26263525939993, 9.6459...",sub-C54-3
4,1,657.1827113056797,1025.5346591258096,L,C,M,C55-3,"[26.020421901161313, 46.18048430372056, 13.711...",sub-C55-3
5,7,"[455.24233996428916, 509.9466847724346, 287.73...",1838.3739820401397,L,C,M,C57-3,6.81653463390343,sub-C57-3


In [9]:
df.id_sub = "sub-" + df.sub_sam

In [ ]:
# number of fascicles

# Dataset
object = Objects(
    id="N:dataset:55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# Domain
unit = Units(
    label="unitless",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/micrometer",
)
aspect = Aspects(
    label="count",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)
# focus of measurement
desc_inst = DescriptorsInst(
    label="fascicle",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    # description=None,
)
# how to calculate unit
desc_quant = DescriptorsQuant(
    shape="scalar",
    label="count of fascicles",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

# Objects
address = Addresses(
    addr_type="constant",
    value_type="single",
)
obj_desc_inst = ObjDescInst(
    objects=object,
    descriptors_inst=desc_inst,
    addresses_field=address,
)
obj_desc_quant = ObjDescQuant(
    objects=object,
    descriptors_quant=desc_quant,
    addresses_field=address,
)

back_populate_tables(session, obj_desc_inst)
back_populate_tables(session, obj_desc_quant)

from sqlalchemy.orm.exc import NoResultFound

for i, row in df.iterrows():
    try:
        # Check if the entry already exists
        existing_values_inst = (
            session.query(ValuesInst)
            .filter_by(id_sub=row.id_sub, id_formal=row.id_sub, objects=object)
            .one()
        )
        values_inst = existing_values_inst
    except NoResultFound:
        # If not found, create a new entry
        values_inst = ValuesInst(
            type="subject",
            id_sub=row.id_sub,
            id_formal=row.id_sub,
            local_identifier=None,
            objects=object,
        )
        session.add(values_inst)
        session.commit()

    values_quant = ValuesQuant(
        value=int(row["NFasc"]),
        objects=object,
        orig_value=None,
        orig_units=None,
        descriptors_inst=desc_inst,
        descriptors_quant=desc_quant,
        values_inst=values_inst,
        value_blob=row["NFasc"],
    )

    back_populate_tables(session, values_quant)
# # values to iterate through
# values_inst = ValuesInst(
#     type="sample",
#     id_sub="sub-",
#     id_formal="",
#     local_identifier=None,
#     id_sam="",
#     # descriptors_inst=desc_inst,
#     # dataset=object.id,
#     objects=object,
# )
# values_quant = ValuesQuant(
#     value=0.5,
#     objects=object,
#     orig_value=None,
#     orig_units=None,
#     descriptors_inst=desc_inst,
#     descriptors_quant=desc_quant,
#     values_inst=values_inst,
#     value_blob={"key1": "value1"},
# )

# back_populate_tables(session, values_quant)

populated {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
object <class 'quantdb.ingest_tabular.models.Addresses'>
filter using {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
is existing? <quantdb.ingest_tabular.models.Addresses object at 0x156c97ef0>
after update {'id': 2, 'addr_type': 'constant', 'value_type': 'single', 'addr_field': 'literally this value right here'}
populated {'id': 22, 'label': 'fascicle', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/fascicle'}
object <class 'quantdb.ingest_tabular.models.DescriptorsInst'>
filter using {'id': 22, 'label': 'fascicle', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/fascicle'}
is existing? <quantdb.ingest_tabular.models.DescriptorsInst object at 0x156c8e780>
after update {'id': 22, 'label': 'fascicle', 'iri': 'https://uri.interlex.org/tgbugs/uris/readable/quantdb/clas

AssertionError: Dependency rule on column 'descriptors_quant.id' tried to blank-out primary key column 'obj_desc_quant.desc_quant' on instance '<ObjDescQuant at 0x15746c7a0>'

In [ ]:
# NFac

# Dataset
object = Objects(
    id="N:dataset:55c5b69c-a5b8-4881-a105-e4048af26fa5",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

# Domain
unit = Units(
    label="unitless",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/units/micrometer",
)
aspect = Aspects(
    label="count",
    description=None,
    iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/count",
)
# focus of measurement
desc_inst = DescriptorsInst(
    label="fascicle",
    # iri="https://uri.interlex.org/tgbugs/uris/readable/quantdb/classes/nerve/volume-test",
    # description=None,
)
# how to calculate unit
desc_quant = DescriptorsQuant(
    shape="scalar",
    label="count of fascicles",
    aggregation_type="instance",
    description=None,
    curator_note=None,
    aspects=aspect,
    descriptors_inst=desc_inst,
)

# Objects
address = Addresses(
    addr_type="constant",
    value_type="single",
)
obj_desc_inst = ObjDescInst(
    objects=object,
    descriptors_inst=desc_inst,
    addresses_field=address,
)
obj_desc_quant = ObjDescQuant(
    objects=object,
    descriptors_quant=desc_quant,
    addresses_field=address,
)

back_populate_tables(session, obj_desc_inst)
back_populate_tables(session, obj_desc_quant)

from sqlalchemy.orm.exc import NoResultFound

for i, row in df.iterrows():
    try:
        # Check if the entry already exists
        existing_values_inst = (
            session.query(ValuesInst)
            .filter_by(id_sub=row.id_sub, id_formal=row.id_sub, objects=object)
            .one()
        )
        values_inst = existing_values_inst
    except NoResultFound:
        # If not found, create a new entry
        values_inst = ValuesInst(
            type="subject",
            id_sub=row.id_sub,
            id_formal=row.id_sub,
            local_identifier=None,
            objects=object,
        )
        session.add(values_inst)
        session.commit()

    values_quant = ValuesQuant(
        value=int(row["NFasc"]),
        objects=object,
        orig_value=None,
        orig_units=None,
        descriptors_inst=desc_inst,
        descriptors_quant=desc_quant,
        values_inst=values_inst,
        value_blob=row["NFasc"],
    )

    back_populate_tables(session, values_quant)

# dream ingest

In [ ]:
# ingest.insert_descriptors_quant(
#     descriptor_quants=[
#         DescriptorQuant(
#             label="fascicle cross section diameter um min",
#             domain=26,  # "fiber-cross-section" # TODO: label to ID
#             aspect=5,  # "diameter",  # TODO: label to ID
#             unit=4,  #  pixel",  # TODO: label to ID
#             aggregation_type="min",
#         ),
#     ]
# )

# # dream ingest
# csv = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
# ingest.insert_descriptors_quant(**csv.to_dict(orient="records"))